In [ ]:
cd /workspace/sunggu/0.Challenge/RSNA2023/

In [ ]:
!nvidia-smi
import os
os.environ["CUDA_DEVICE_ORDER"]     =  'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"]  =  '2'
print("CPU 갯수 = ", os.cpu_count())

In [ ]:
# !pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu117
# !pip freeze > /workspace/sunggu/0.Challenge/requirements.txt
!pip install -r requirements.txt
!pip install -U nibabel

# 0. Fix Seed

In [ ]:
import random
import numpy as np
import torch

# 시드(seed) 설정
seed = 42

# Python의 random 모듈 시드 설정
random.seed(seed)

# Numpy 시드 설정
np.random.seed(seed)

# Torch 시드 설정
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# 1. Dataset

In [ ]:
import os
import cv2
import pandas as pd
import pydicom
import skimage
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from pydicom.pixel_data_handlers.util import apply_modality_lut, apply_voi_lut


def list_sort_nicely(l):
    def convert(text): return int(text) if text.isdigit() else text
    def alphanum_key(key): return [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key=alphanum_key)

class RSNA_Dataset(Dataset):
    def __init__(self, data_dir, csv_file, target_class, mode="train"):
        self.data_dir = data_dir
        self.csv_file = csv_file
        self.target_class = target_class
        self.mode = mode

        # CSV 파일에서 데이터 로드
        self.df = pd.read_csv(os.path.join(self.data_dir, self.csv_file))
        
        # 해당하는 모드에 따라 데이터 필터링
        self.filtered_df = self.df[self.df['mode'] == self.mode].reset_index(drop=True)

        # Albumentations 변환 함수 정의
        self.transforms = self.get_transforms()

    def __len__(self):
        return len(self.filtered_df)

    def __getitem__(self, idx):
        # 이미지 경로 및 라벨 가져오기
        img_path = self.filtered_df['path'].iloc[idx]
        label = self.filtered_df[self.target_class].iloc[idx]

        # DICOM 파일 읽기 및 처리
        dcm_data = pydicom.dcmread(img_path)
        temp_img = apply_modality_lut(dcm_data.pixel_array, dcm_data)
        image = apply_voi_lut(temp_img, dcm_data)

        # 라벨을 Tensor로 변환
        label = torch.tensor(label).float().unsqueeze(0)

        # 채널 차원 추가
        image = np.expand_dims(image, axis=-1)

        # Albumentations 변환 수행
        transformed = self.transforms(image=image)
        image = transformed['image']

        return image, label

    def get_transforms(self):
        if self.mode == "train":
            return A.Compose([
                A.Resize(224, 224),
                A.Lambda(image=self.min_max_normalization, always_apply=True),
                A.Lambda(image=self.change_to_uint8, always_apply=True),
                A.CLAHE(clip_limit=2.0, tile_grid_size=(8, 8), always_apply=True),
                A.Lambda(image=self.change_to_float32, always_apply=True),
                
                # Augmentation
                A.HorizontalFlip(p=0.5),
                A.RandomBrightnessContrast(p=0.5),
                A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=30, p=0.5),
                A.InvertImg(p=0.5),

                # Normalize
                A.Lambda(image=self.min_max_normalization, always_apply=True),
                A.Normalize(max_pixel_value=1.0, mean=0.5, std=0.5),
                ToTensorV2()
            ])
        else:
            return A.Compose([
                A.Resize(224, 224),
                A.Lambda(image=self.min_max_normalization, always_apply=True),
                A.Lambda(image=self.change_to_uint8, always_apply=True),
                A.Lambda(image=self.fixed_clahe, always_apply=True),
                A.Lambda(image=self.change_to_float32, always_apply=True),

                # Normalize
                A.Lambda(image=self.min_max_normalization, always_apply=True),
                A.Normalize(max_pixel_value=1.0, mean=0.5, std=0.5),
                ToTensorV2()
            ])

    def min_max_normalization(self, image, **kwargs):
        if np.unique(image).size == 1:
            return image    
        image = image.astype('float32')
        image -= image.min()
        image /= image.max()
        return image

    def fixed_clahe(self, image, **kwargs):
        clahe_mat = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        return clahe_mat.apply(image)

    def change_to_uint8(self, image, **kwargs):
        return skimage.util.img_as_ubyte(image)

    def change_to_float32(self, image, **kwargs):
        return skimage.util.img_as_float32(image)


In [ ]:
data_dir = '/workspace/sunggu/0.Challenge/Med_tutorial_ChatGPT/dataset'
csv_file = 'rsna_data.csv'
target_class = 'cancer'

# 학습 데이터셋 생성
train_dataset = RSNA_Dataset(data_dir, csv_file, target_class, mode='train')

# 검증 데이터셋 생성
valid_dataset = RSNA_Dataset(data_dir, csv_file, target_class, mode='valid')

# DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=200, shuffle=True, num_workers=40)
valid_loader = DataLoader(valid_dataset, batch_size=1, shuffle=False, num_workers=4)

# 2. Network

In [2]:
import torch.nn as nn
import torchvision

class ResNet50(nn.Module):
    def __init__(self, pretrained=True):
        super(ResNet50, self).__init__()
        self.model = torchvision.models.resnet50(pretrained=pretrained)
        self.model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.model.fc = nn.Linear(2048, 1)
        
    def forward(self, x):
        return self.model(x)


In [3]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

model = ResNet50()
num_params = count_parameters(model)
print("Number of learnable parameters: ", num_params)

/home/sunggu/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/sunggu/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Number of learnable parameters:  23503809


# 3. Loss

In [ ]:
# Define the loss function

# 4. Optimizer

In [ ]:
# Define optimizer and scheduler

from optimizers import get_optimizer
optimizer = get_optimizer(name='adamw', model=model, lr=1e-4)

# 5. LR scheduler

In [ ]:
# Define learning rate scheduler

# Use a learning rate scheduler to reduce the learning rate over time

# 6. Check the resume point

In [ ]:
from utils import load_checkpoint

start_epoch     = 0
total_epoch     = 1000
checkpoint_path = '/workspace/sunggu/0.Challenge/RSNA2023/checkpoints/230530_ResNet50'
save_dir        = '/workspace/sunggu/0.Challenge/RSNA2023/predictions/230530_ResNet50'

# make folder if not exist
os.makedirs(checkpoint_path, exist_ok =True)
os.makedirs(save_dir, exist_ok =True)

# Resume
print("Loading... Resume")
start_epoch, best_loss, model, optimizer, scheduler = load_checkpoint(model, optimizer, scheduler, filename='checkpoint.pth')

# Optimizer Error fix...!
for state in optimizer.state.values():
    for k, v in state.items():
        if torch.is_tensor(v):
            state[k] = v.cuda()

# 7. Using the DataParallel for multi-gpu training

In [ ]:
# Using the DataParallel for multi-gpu training

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = torch.nn.DataParallel(model)
model.to(device)

# 8. Metric

In [ ]:
# Define metrics

from sklearn.metrics import roc_auc_score, accuracy_score, f1_score

# sensitivity_score, specificity_score
def sensitivity_score(y_true, y_pred):
    y_true_pos = y_true[y_true == 1.0]
    y_pred_pos = y_pred[y_true == 1.0]

    # 길이 확인
    if len(y_true) != len(y_pred):
        print("Error: y_true와 y_pred의 길이가 다릅니다.")
    # 리스트 또는 배열 형태 확인
    if not isinstance(y_true, (list, np.ndarray)):
        print("Error: y_true는 리스트나 배열 형태로 전달되어야 합니다.")
    if not isinstance(y_pred, (list, np.ndarray)):
        print("Error: y_pred는 리스트나 배열 형태로 전달되어야 합니다.")

    return accuracy_score(y_true_pos, y_pred_pos)

def specificity_score(y_true, y_pred):
    y_true_neg = y_true[y_true == 0.0]
    y_pred_neg = y_pred[y_true == 0.0]
    
    # 길이 확인
    if len(y_true) != len(y_pred):
        print("Error: y_true와 y_pred의 길이가 다릅니다.")
    # 리스트 또는 배열 형태 확인
    if not isinstance(y_true, (list, np.ndarray)):
        print("Error: y_true는 리스트나 배열 형태로 전달되어야 합니다.")
    if not isinstance(y_pred, (list, np.ndarray)):
        print("Error: y_pred는 리스트나 배열 형태로 전달되어야 합니다.")

    return accuracy_score(y_true_neg, y_pred_neg)



# 9. Training & Validation Loop

In [ ]:
import time
import math
import json
from tqdm import tqdm
import datetime
import matplotlib.pyplot as plt
from collections import defaultdict

class AverageMeter:
    def __init__(self, **kwargs):
        self.reset()

    def reset(self):
        self.data = defaultdict(lambda: {'sum': 0, 'count': 0})

    def update(self, key, value, n):
        self.data[key]['sum']   += value * n
        self.data[key]['count'] += n
    
    def average(self):
        return {k: v['sum'] / v['count'] for k, v in self.data.items()}

In [ ]:
# Define train the network
def train_loop_fn(train_loader, model, criterion, optimizer, device, epoch):
    model.train()
    metric_logger  = AverageMeter()      # Initialize the network
    epoch_iterator = tqdm(train_loader, desc="Training (X / X Steps) (loss=X.X)", dynamic_ncols=True, total=len(train_loader))

    for step, batch_data in enumerate(epoch_iterator):

        image, target = batch_data
        image, target = image.to(device), target.to(device)

        logit = model(image)     # Forward pass
        loss  = criterion(logit, target)  # Compute the loss
        loss_value = loss.item()

        if not math.isfinite(loss_value):
            print("Loss is {}, stopping training".format(loss_value))

        optimizer.zero_grad()  # Clear the gradients
        loss.backward()   # Backward pass
        optimizer.step()    # Update the weights

        # Log the training and validation performance
        metric_logger.update(key='train_loss', value=loss_value, n=image.shape[0])
        metric_logger.update(key='lr', value=optimizer.param_groups[0]["lr"], n=1)    
        epoch_iterator.set_description("Training: Epochs %d (%d / %d Steps), (train_loss=%2.5f)" % (epoch, step, len(train_loader), loss_value))

    return {k: round(v, 7) for k, v in metric_logger.average().items()}

@torch.no_grad()
def valid_loop_fn(valid_loader, model, criterion, device, epoch, save_dir):
    model.eval()
    metric_logger  = AverageMeter()
    epoch_iterator = tqdm(valid_loader, desc="Validating (X / X Steps) (loss=X.X)", dynamic_ncols=True, total=len(valid_loader))

    preds = [] # must be 1d list or array
    gts   = [] # must be 1d list or array
    for step, batch_data in enumerate(epoch_iterator):
        image, target = batch_data
        image, target = image.to(device), target.to(device)
        logit = model(image)
        loss  = criterion(logit, target)
        loss_value = loss.item()

        if not math.isfinite(loss_value):
            print("Loss is {}, stopping training".format(loss_value))

        metric_logger.update(key='valid_loss', value=loss_value, n=image.shape[0])
        epoch_iterator.set_description("Validating: Epochs %d (%d / %d Steps), (valid_loss=%2.5f)" % (epoch, step, len(valid_loader), loss_value))

        # post-processing
        preds.append(logit.sigmoid().squeeze().detach().cpu().numpy())
        gts.append(target.squeeze().detach().cpu().numpy())     

    # Metric Calculation
    preds = np.array(preds)
    gts   = np.array(gts)

        # AUC
    auc = roc_auc_score(gts, preds)
    metric_logger.update(key='AUC', value=auc.item(), n=image.shape[0])
    
        # F1
    f1 = f1_score(gts, np.round(preds))
    metric_logger.update(key='F1', value=f1.item(), n=image.shape[0])

        # Accuracy
    acc = accuracy_score(gts, np.round(preds))
    metric_logger.update(key='Acc', value=acc.item(), n=image.shape[0])

        # Sensitivity
    sen = sensitivity_score(gts, np.round(preds))
    metric_logger.update(key='Sen', value=sen.item(), n=image.shape[0])        

        # Specificity
    spe = specificity_score(gts, np.round(preds))
    metric_logger.update(key='Spe', value=spe.item(), n=image.shape[0])                

    return {k: round(v, 7) for k, v in metric_logger.average().items()}

# 9. Test Result

- ## Log 조사

In [ ]:
import glob
import numpy as np
import matplotlib.pyplot as plt

def read_log(path):
    log_list = []
    lines = open(path, 'r').read().splitlines() 
    for line in lines:
        log_list.append(eval(line))
    return  log_list

# log 파일을 읽어옵니다.
log_list = read_log(path = '/workspace/sunggu/0.Challenge/RSNA2023/checkpoints/230530_ResNet50/log.txt')

# 결과를 저장할 딕셔너리를 생성합니다.
result_dict = {}
for key in log_list[0].keys():
    result_dict[key] = [log[key] for log in log_list]

# 그래프를 생성합니다.
fig, axs = plt.subplots(len(result_dict.keys()), 1, figsize=(10, len(result_dict.keys())*5))

for idx, key in enumerate(result_dict.keys()):
    axs[idx].plot(result_dict[key])
    axs[idx].set_title(key)
    print("###########################################################")
    print("Metric  = ", key)
    
    if "loss" in key:
        print("Argsort = ", np.argsort(result_dict[key])[:5])
        print("Value   = ", np.array(result_dict[key])[np.argsort(result_dict[key])[:5]])
    else:
        print("Argsort = ", np.argsort(result_dict[key])[::-1][:5])
        print("Value   = ", np.array(result_dict[key])[np.argsort(result_dict[key])[::-1][:5]])

plt.tight_layout()
plt.show()


- ## Testing

In [ ]:
cd /workspace/sunggu/0.Challenge/RSNA2023/

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]     =  'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"]  =  '3'

In [ ]:
import os
import numpy as np 
import pandas as pd 
import glob
import torch

from create_datasets.RSNA import RSNA_Dataset_TEST
from torch.utils.data import DataLoader

# 0. Check my computer's cpu core num
print("CPU 갯수 = ", os.cpu_count())

# 1. Create Dataset
test_dataset  = RSNA_Dataset_TEST()

# 2. Create DataLoader
test_loader  = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=32)


In [ ]:
from create_models import get_model
    
model = get_model(name='Resnet50').to('cuda')

In [ ]:
# Resume checkpoint
import torch

print("Loading... Resume")
checkpoint = torch.load('/workspace/sunggu/0.Challenge/RSNA2023/checkpoints/230530_ResNet50/epoch_25_checkpoint.pth', map_location='cpu')
model.load_state_dict(checkpoint['model_state_dict'])        
device = 'cuda'

In [ ]:
from engine import test_loop_fn

preds, ids = test_loop_fn(test_loader, model, device)

In [ ]:
squeeze_ids = [i[0] for i in ids]
squeeze_ids

In [ ]:
preds

# Submission for Kaggle

In [ ]:
# read csv
import pandas as pd
df = pd.DataFrame()

df['prediction_id'] = squeeze_ids
df['cancer'] = preds

In [ ]:
df

In [ ]:
grouped_mean = df.groupby('prediction_id').mean()
grouped_mean 

In [ ]:
reset_df = grouped_mean.reset_index()

In [ ]:
reset_df

In [ ]:
# save csv
reset_df.to_csv('/workspace/sunggu/0.Challenge/RSNA2023/dataset/rsna-breast-cancer-detection/sample_submission_2.csv', index=False)